# 1. Getting the Data

First we will scrape policies from the gov.ie website.

In your command line, ``cd`` into this repository.

``cd`` into the ``policy_scraping`` task directory, then ``cd`` again into the ``policy_scraping`` scrapy environment.

In [1]:
import os
cwd = os.getcwd() # should be base directory of repository
os.chdir(cwd+"/policy_scraping/policy_scraping")

Run ``scrapy crawl goviefor -O ../outputs/goviefor.json`` (or you can change the -O argument to whatever you would prefer the output file information to be).

This command will generate a json containing the metadata about all the policies as well as download all files to the same outputs directory under ``forestry/full``.

In [ ]:
!! scrapy crawl goviefor -O ../outputs/goviefor.json

Next we will consolidate the metadata and text of the policy PDFs into one dictionary.

In [2]:
os.chdir(cwd) # back to base directory
import json
from populate_corpora.pdfs_to_jsons import scrp_itm_to_fulltxt
FILE_DIR= cwd+"/policy_scraping/policy_scraping/outputs" # or whatever output directory you gave the scraper for its output json

In [ ]:
with open(cwd+"/policy_scraping/outputs/goviefor.json","r", encoding="utf-8") as f:
    metad = json.load(f)
pdf_dict = scrp_itm_to_fulltxt(metad, FILE_DIR+"/forestry/full")

If you have your own collection of pdfs to process and don't have a metadata file, you can use this next function on just the file directory.

In [ ]:
from populate_corpora.pdfs_to_jsons import pdfs_to_txt_dct
pdf_dict = pdfs_to_txt_dct(FILE_DIR+"/forestry/full") # or whatever your policy directory is

For the purposes of this project, we only want the texts of the PDFs in cleaned sentences anyways. So we'll go ahead and extract/clean those sentences, then load them into the dictionary format that doccano (labeling platform) uses. Finally, if we want, we can use a simple keyword search to prelabel some of the sentences with a "incentive class mention" label.

In [ ]:
import nltk
from populate_corpora.data_cleaning import get_clean_text_sents, format_sents_for_doccano, prelabeling
EN_TOKENIZER = nltk.data.load("tokenizers/punkt/english.pickle") # need tokenizer for our text cleaning
clean_sents= get_clean_text_sents(pdf_dict, EN_TOKENIZER)
doccano_dict = format_sents_for_doccano(clean_sents)
prelab_doccano_dict = prelabeling(doccano_dict)

Now we can download this dictionary as a json to import into our doccano instance for labeling.

In [ ]:
with open(cwd+"/populate_corpora/outputs/ready_to_label.json", 'w', encoding="utf-8") as outfile:
    json.dump(prelab_doccano_dict, outfile, ensure_ascii=False, indent=4)

# 2. Labeling the Data

## Augmentation via Sentence Similarity Search

We also need to make a new human-in-the-loop dataset using by doing sentence similarity searches with predefined queries. We have five queries for each label.

In [ ]:
with open(cwd+"/populate_corpora/outputs/ready_to_label.json","r", encoding="utf-8") as f:
    prelab_doccano_dict = json.load(f)

In [ ]:
from populate_corpora.query_augment import run_embedder, run_queries, QUERIES_DCT
from populate_corpora.data_cleaning import dcno_to_only_sents

# loading all sentences, not just the labeled ones
# or reload cwd+"/populate_corpora/outputs/ready_to_label.json"
all_sents = dcno_to_only_sents(prelab_doccano_dict) 
embs, s_sentences, model = run_embedder(sample=False, dev='cuda', data=all_sents, unique=True)
# uses our queries dictionary, but obvs you can make your own
qry_dct = run_queries(embs, s_sentences, model, qry_dct=QUERIES_DCT, dev='cuda', sim_thresh=0.5, res_lim=1000)

NameError: name 'prelab_doccano_dict' is not defined

Now we'll parse the results and create a dataset of sentences labeled by the query process, but we first need to filter them to only include sentences found by at least 4/5 queries for each label.

In [ ]:
from populate_corpora.query_augment import consolidate_sents, crossref_sents
lbl_qry_dct = consolidate_sents(qry_dct, QUERIES_DCT)
filt_qry_dct = crossref_sents(lbl_qry_dct, 4)
qry_rs_dataset = [{'text': sent, 'label': lbl} for lbl in list(filt_qry_dct) for sent in filt_qry_dct[lbl]]

In [ ]:
with open(cwd+"/populate_corpora/outputs/augmented_to_label.json", 'w', encoding="utf-8") as outfile:
    json.dump(qry_rs_dataset, outfile, ensure_ascii=False, indent=4)

## External Annotation

We used a doccano instance for our labeling, but we also had to do some data validation with an external annotator. This section generates a subset for a labeler from the hand-labeled dataset.

In [5]:
from populate_corpora.annotators import resample_forannot
from populate_corpora.data_cleaning import dcno_to_sentlab, remove_duplicates, group_duplicates
with open(cwd+"/inputs/19Jan25_firstdatarev.json","r", encoding="utf-8") as f:
    dcno_json = json.load(f)
with open(cwd+"/inputs/27Jan25_query_checked.json","r", encoding="utf-8") as f:
    qry_json = json.load(f)
sents1, labels1 = dcno_to_sentlab(dcno_json)
sents2, labels2 = dcno_to_sentlab(qry_json)
sents3 = sents1+sents2
labels3 = labels1+labels2
all_sents, all_labs = remove_duplicates(group_duplicates(sents3,labels3,thresh=90))

1419 groups found with a threshold of 90
Sanity check: 1419 sentences and 1419 labels


In [10]:
ann_sents, ann_labels = resample_forannot(all_sents, all_labs, 0.3, 0.5)
print(round(len(ann_sents)/len(all_sents), 3))

Counter({'Non-Incentive': 1150, 'Supplies': 81, 'Technical_assistance': 75, 'Direct_payment': 62, 'Fine': 23, 'Credit': 19, 'Tax_deduction': 9})
Counter({'Supplies': 81, 'Technical_assistance': 75, 'Direct_payment': 62, 'Fine': 23, 'Credit': 19, 'Tax_deduction': 9})
Counter({'Supplies': 24, 'Technical_assistance': 22, 'Direct_payment': 19, 'Fine': 7, 'Credit': 6, 'Tax_deduction': 3})
Counter({'Non-Incentive': 81, 'Supplies': 24, 'Technical_assistance': 22, 'Direct_payment': 19, 'Fine': 7, 'Credit': 6, 'Tax_deduction': 3})
Should be true: True
162 162
0.114


In [13]:
import random
ann_frame = [{'text':ann_sents[i], 'label':[]} for i in range(len(ann_labels))]
random.shuffle(ann_frame)
with open(cwd+"/inputs/subsample.json", 'w', encoding="utf-8") as outfile:
    json.dump(ann_frame, outfile, ensure_ascii=False, indent=4)
val_frame = [{'text':ann_sents[i], 'label':ann_labels[i]} for i in range(len(ann_labels))]
with open(cwd+"/inputs/subsample_key.json", 'w', encoding="utf-8") as outfile:
    json.dump(val_frame, outfile, ensure_ascii=False, indent=4)

Now let's check the inter-annotator agreement.

In [ ]:
with open(cwd+"/inputs/annotation_odon.json","r", encoding="utf-8") as f: #our hand-labeled dataset
    ann_json = json.load(f)

sents_a, labels_a = dcno_to_sentlab(ann_json)
# correct labels
swap_labs = {'non-incentive':'Non-Incentive', 'fine':'Fine', 'tax deduction':'Tax_deduction', 'credit':'Credit', 'direct payment':'Direct_payment', 'supplies':'Supplies', 'technical assistance':'Technical_assistance'}
sents_a2, labels_a2 = [], []
for i, lab in enumerate(labels_a):
  try:
    labels_a2.append(swap_labs[lab])
    sents_a2.append(sents_a[i])
  except:
    pass

In [ ]:
from populate_corpora.annotators import get_common_sentlabs, all_to_bin, all_to_sharedmc
from sklearn.metrics import cohen_kappa_score

s_sents, labels_sc, labels_sa = get_common_sentlabs(sents_d, labels_d, sents_a2, labels_a2)
#print(f"All: {cohen_kappa_score(labels_sc, labels_sa)} for {len(labels_sc)} entries")

labs_binc, labs_bina = all_to_bin(labels_sc), all_to_bin(labels_sa)
print(f"Binary: {cohen_kappa_score(labs_binc, labs_bina)} for {len(labs_binc)} entries")

mclabsc, mclaba = all_to_sharedmc(labels_sc, labels_sa, labs_binc, labs_bina)
print(f"Multiclass: {cohen_kappa_score(mclabsc, mclaba)} for {len(mclabsc)} entries")

All: 0.7707100591715976 for 62 entries
Binary: 0.7114788004136505 for 62 entries
Multiclass: 0.9534883720930233 for 26 entries


## Consolidation into Final Dataset

We now have all of our data labeled, so it is time to create a final dataset broken into training and testing sets.

In [3]:
from populate_corpora.data_cleaning import dcno_to_sentlab
from classifier.run_classifiers import group_duplicates, remove_duplicates

with open(cwd+"/inputs/19Jan25_firstdatarev.json","r", encoding="utf-8") as f: #our hand-labeled dataset
    dcno_json = json.load(f)
with open(cwd+"/inputs/27Jan25_query_checked.json","r", encoding="utf-8") as f: #our human-in-the-loop dataset
    aug_json = json.load(f)

sents_d, labels_d = dcno_to_sentlab(dcno_json)
sents_a, labels_a = dcno_to_sentlab(aug_json)

all_sents = sents_d+sents_a
all_labs = labels_d+labels_a
sentences, labels = remove_duplicates(group_duplicates(all_sents,all_labs,thresh=90))

1419 groups found with a threshold of 90
Sanity check: 1419 sentences and 1419 labels


At this point, we have a set of all the sentences and labels, including non-incentives and classified incentive types. Now we'll generate new sets for each binary and multiclass classifications, binary containing only labels of incentive/non-incentive, and multiclass including only the classified incentive types.

In [ ]:
from classifier.run_classifiers import gen_bn_sentlab, gen_mc_sentlab, generate_dataset

bn_sents, bn_labels = gen_bn_sentlab(sentences, labels)
mc_sents, mc_labels = gen_mc_sentlab(sentences, labels)

r = 9
ts = 0.2

bn_ds, bn_name = generate_dataset(bn_sents, bn_labels,test_split=ts, r_state=r)
mc_ds, mc_name = generate_dataset(mc_sents,mc_labels,test_split=ts, r_state=r)

Sanity Check: 269 incentive sentences and 1150 non-incentive sentences
Incentives: 0.18957011980267793; Non-Incentives: 0.8104298801973221
Sanity Check: 269 incentive sentences and 269 incentive labels


In [8]:
with open(cwd+f"/inputs/bn_dataset{bn_name}.json", 'w', encoding="utf-8") as outfile:
    json.dump(bn_ds, outfile, ensure_ascii=False, indent=4)

with open(cwd+f"/inputs/mc_dataset{mc_name}.json", 'w', encoding="utf-8") as outfile:
    json.dump(mc_ds, outfile, ensure_ascii=False, indent=4)

# 3. Fine-Tuning Our Models

Now that we have our datasets, it's time to fine-tune our models.

{'train': [{'text': 'Importantly, it may be combined with grant support such as that provided through the Targeted Agriculture Modernisation Scheme (TAMS), ensuring farmers have access to funding for investment projec ts, promoting the competitiveness and viability of Irish farms.',
   'label': 'incentive'},
  {'text': '\uf0b7 Employer’s PRSI: Galmoy paid Pay-Related Social Insurance (PRSI) on behalf of their employees.',
   'label': 'non-incentive'},
  {'text': 'The Future Growth Loan Scheme is a long -term loan (7 -10 years) that is offered by the SBCI with the support of the Department o f Enterprise, Trade and Employment, the Department of Agriculture, Food and the Marine, the European Investment Bank and the European Investment Fund (EIF).',
   'label': 'incentive'},
  {'text': 'The goal of the scheme is to provide support for farmers to invest in productive equipment for their farms.',
   'label': 'incentive'},
  {'text': 'Note the following: ➢ If faced with a particularly sensit